## Clean Evaluation

dataset
- cifar10
- svhn

model_name
- resnet18
- [soft-pruning]
    - resnet18_sfP
    - resnet18_sfP-mixup
    - resnet18_sfP-cutout
- [post-training pruning]
    - resnet18_PX_0.Y
        - X = 2, 3, 4
        - Y = 3, 6, 9

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import json

from utils_model import *

dir_results = './results/'

In [ ]:
model_names =  ['resnet18', 'resnet18_sfP', 'resnet18_sfP-mixup', 'resnet18_sfP-cutout']
for layer in [2, 3, 4]:
    for prune_pct in ['0.3', '0.6', '0.9']:
        model_names.append('resnet18_P%i-%s' % (layer, prune_pct))
model_names

In [11]:
df = pd.read_csv(dir_results + 'eval_clean.csv', index_col = 0)

In [12]:
df[df['dataset'] == 'cifar10']

,model_name,dataset,top1acc,top5acc,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
0,resnet18,cifar10,0.9402,0.9981,0.1024,0.1012,0.0988,0.0987,0.1035,0.0988,0.0996,0.0981,0.1001,0.0988
1,resnet18_sfP,cifar10,0.7951,0.9881,0.0986,0.0970,0.0999,0.1448,0.0769,0.1042,0.0916,0.0870,0.1027,0.0973
2,resnet18_sfP-mixup,cifar10,0.8609,0.9863,0.1106,0.1038,0.1022,0.0990,0.0912,0.0937,0.0995,0.1002,0.1000,0.0998
3,resnet18_sfP-cutout,cifar10,0.8354,0.9915,0.0960,0.1008,0.1159,0.1197,0.0756,0.0865,0.0949,0.1045,0.0955,0.1106
8,resnet18_P2-0.3,cifar10,0.9165,0.9970,0.1001,0.0995,0.0997,0.1035,0.1114,0.0849,0.1021,0.0943,0.1095,0.0950
9,resnet18_P2-0.6,cifar10,0.7655,0.9797,0.0922,0.0700,0.0723,0.0869,0.1567,0.0548,0.1569,0.0850,0.1369,0.0883
10,resnet18_P2-0.9,cifar10,0.1477,0.6342,0.0027,0.0000,0.0621,0.0131,0.0003,0.0009,0.8772,0.0000,0.0437,0.0000
11,resnet18_P3-0.3,cifar10,0.9352,0.9980,0.1037,0.1009,0.0967,0.1001,0.1049,0.0954,0.1015,0.0958,0.1010,0.1000
12,resnet18_P3-0.6,cifar10,0.9149,0.9966,0.1075,0.1012,0.0942,0.0992,0.1050,0.0916,0.1106,0.0887,0.1056,0.0964
13,resnet18_P3-0.9,cifar10,0.6101,0.9602,0.1107,0.0933,0.0436,0.3537,0.0639,0.0062,0.1568,0.0459,0.0853,0.0406


In [13]:
df[df['dataset'] == 'svhn']

,model_name,dataset,top1acc,top5acc,label_dist0,label_dist1,label_dist2,label_dist3,label_dist4,label_dist5,label_dist6,label_dist7,label_dist8,label_dist9
4,resnet18,svhn,0.942571,0.993738,0.067571,0.198794,0.160418,0.108597,0.095383,0.090734,0.075330,0.077213,0.062231,0.063729
5,resnet18_sfP,svhn,0.968078,0.996427,0.069683,0.197872,0.156269,0.113399,0.096535,0.091810,0.076790,0.075599,0.061809,0.060234
6,resnet18_sfP-mixup,svhn,0.968155,0.995698,0.069030,0.195029,0.159266,0.109557,0.096919,0.091656,0.078211,0.078711,0.061424,0.060195
7,resnet18_sfP-cutout,svhn,0.971074,0.996658,0.067648,0.194568,0.159957,0.114474,0.096420,0.089083,0.076406,0.077597,0.062846,0.061002
17,resnet18_P2-0.3,svhn,0.936693,0.993470,0.069568,0.194722,0.158075,0.111939,0.095037,0.091234,0.075907,0.077866,0.062346,0.063307
18,resnet18_P2-0.6,svhn,0.895091,0.988745,0.089006,0.188960,0.142555,0.118585,0.086739,0.091195,0.075907,0.078250,0.065035,0.063768
19,resnet18_P2-0.9,svhn,0.376383,0.829057,0.466503,0.116318,0.018900,0.058351,0.009296,0.036686,0.059388,0.021819,0.194107,0.018631
20,resnet18_P3-0.3,svhn,0.936079,0.992778,0.069069,0.193531,0.161532,0.107368,0.094000,0.092694,0.077750,0.078557,0.062923,0.062577
21,resnet18_P3-0.6,svhn,0.915335,0.989590,0.062462,0.192455,0.157306,0.112669,0.087200,0.090811,0.086202,0.082207,0.071566,0.057122
22,resnet18_P3-0.9,svhn,0.496965,0.883720,0.012446,0.110671,0.087277,0.416411,0.012293,0.060963,0.063576,0.011563,0.211624,0.013176
